Goal:

Make a complex net.

Todo.
* Find better orthogonal init

https://github.com/tensorflow/tensorflow/issues/2255

In [1]:
import tensorflow as tf
import functools
import numpy as np
from sklearn.datasets import load_digits
digits = load_digits()
from sklearn.preprocessing import OneHotEncoder
labels = OneHotEncoder().fit_transform(digits.target.reshape((digits.target.shape[0],1))).toarray()

In [2]:
d = 64
batch_size = 64
X = tf.placeholder(tf.float32,shape=(None,batch_size))
x = tf.complex(X,tf.zeros((d,batch_size)))
T = tf.placeholder(tf.float32,shape=(None,labels.shape[1]))

In [3]:
def complex_variable( name, shape):
    re, im = tf.get_variable(name+'_re', shape=shape), tf.get_variable(name+'_im', shape=shape)
    return tf.complex(re,im, name=name)

In [4]:
class Mapping():
    num = 0
    def __init__(self,shape):
        self.name = 'Map' + str(Mapping.num)
        Mapping.num+=1
        self.shape = shape
        with tf.variable_scope(self.name,initializer=tf.contrib.layers.xavier_initializer()):
            self.weights = complex_variable('W',shape)
            self.biases = complex_variable('b',[1,shape[-1]])
    
    def __call__(self,x):
        with tf.name_scope(self.name+'_calc'):
            return tf.matmul(x,self.weights) + self.biases

In [5]:
class LinearNet():
    name = 'LinearNet'
    layers = [Mapping((d,64)),  
              Mapping((64,d))]

In [6]:
NN = LinearNet
y = functools.reduce(lambda x,f:f(x),[x] + NN.layers)
tf.image_summary('Recons',tf.reshape(tf.complex_abs(y),[batch_size,8,8,1]),max_images=10)

def mse(x,y,name='mse'): #is this the right equation to be using?
    with tf.name_scope(name):
        return tf.reduce_mean(tf.reduce_sum(tf.square(tf.complex_abs(x-y)),reduction_indices=[1]))
    
Loss = mse(x,y)
tf.scalar_summary("Loss",Loss)

for var in tf.trainable_variables():
    tf.histogram_summary(var.name,var)

In [7]:
train_step = tf.train.AdamOptimizer(0.001).minimize(Loss)
merged = tf.merge_all_summaries()
saver = tf.train.Saver()

In [8]:
def batch_iter(data, batch_size):
    num_batches = data.shape[0] // batch_size
    
    for i in range(num_batches):
        yield data[i*batch_size:(i+1)*batch_size, ...],labels[i*batch_size:(i+1)*batch_size, ...]

In [9]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    writer = tf.train.SummaryWriter('Summaries/'+NN.name,sess.graph)
    count = 0
    feed = {}
    for e in range(50):
        for i,batch in enumerate(batch_iter(digits.data,batch_size)):
            ims,lab = batch
            feed[x] = ims.reshape((batch_size,d))/255
            count +=1
            sess.run(train_step,feed_dict=feed)
            if i%10==0:
                summaries = sess.run(merged,feed_dict=feed)
                writer.add_summary(summaries,count)
                print('\r Loss {}'.format(sess.run(Loss,feed)),end='')

 Loss 1.095484972000122